In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/drw-crypto-market-prediction/sample_submission.csv
/kaggle/input/drw-crypto-market-prediction/train.parquet
/kaggle/input/drw-crypto-market-prediction/test.parquet


In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr
import polars as pl
import numpy as np
from tqdm import tqdm
from datetime import datetime

# 1 Data

In [3]:
data = pl.read_parquet(
    "/kaggle/input/drw-crypto-market-prediction/train.parquet"
).sort("timestamp", descending = False)
data

bid_qty,ask_qty,buy_qty,sell_qty,volume,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,…,X856,X857,X858,X859,X860,X861,X862,X863,X864,X865,X866,X867,X868,X869,X870,X871,X872,X873,X874,X875,X876,X877,X878,X879,X880,X881,X882,X883,X884,X885,X886,X887,X888,X889,X890,label,timestamp
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[ns]
15.283,8.425,176.405,44.984,221.389,0.121263,-0.41769,0.005399,0.125948,0.058359,0.027359,0.03578,0.068219,1.034825,-0.029575,0.327805,0.485823,0.668596,0.617389,0.770037,0.857631,1.754456,0.572503,0.883229,0.58567,0.816321,0.529973,0.508244,0.448616,1.341892,1.406392,0.953631,1.183991,1.474789,0.774389,0.660586,0.269043,…,-0.216525,0.200508,0.492433,-0.51249,0.541286,-0.336399,-1.027483,0.21857,0.0,1.728155,0.62414,0.0,-0.051211,0.0,0.0,0.0,0.0,0.691754,0.242124,2.096157,3.369195,0.244667,0.286611,0.722679,0.901931,1.000007,1.925423,1.847943,0.005676,0.190791,0.369691,0.37763,0.210153,0.159183,0.530636,0.562539,2023-03-01 00:00:00
38.59,2.336,525.846,321.95,847.796,0.302841,-0.049576,0.356667,0.481087,0.237954,0.208359,0.217057,0.249624,0.948694,-0.183488,0.150526,0.308421,0.492232,0.529787,0.682958,0.770965,1.686504,0.273357,0.591695,0.442391,0.674792,0.460741,0.439681,0.380399,1.304113,1.003783,0.776628,1.015943,1.312735,0.696895,0.584217,0.231104,…,-0.180112,0.213252,0.479806,-0.180527,0.450331,-0.31915,-1.024055,0.088014,0.0,1.665698,0.622775,0.0,-0.079621,0.0,0.0,0.0,0.0,0.691665,0.242091,2.46103,4.127584,0.321394,0.31246,0.746452,0.912371,1.003153,1.928569,1.849468,0.005227,0.18466,0.363642,0.374515,0.209573,0.158963,0.530269,0.533686,2023-03-01 00:01:00
0.442,60.25,159.227,136.369,295.596,0.167462,-0.291212,0.083138,0.206881,0.101727,0.072778,0.081564,0.114166,0.896459,-0.261779,0.044571,0.200608,0.384558,0.476229,0.629848,0.718232,1.656707,0.140156,0.457268,0.376524,0.610116,0.429751,0.409316,0.350359,1.28325,0.760801,0.670816,0.917205,1.219124,0.653355,0.541739,0.210095,…,-0.265966,0.191734,0.440207,-0.108209,0.420681,-0.316953,-1.024056,-0.147363,0.0,1.666893,0.621414,0.0,-0.080427,0.0,0.0,0.0,0.0,0.691674,0.242093,2.493249,4.182112,0.326701,0.314636,0.746681,0.911129,1.002502,1.928047,1.849282,0.004796,0.178719,0.357689,0.371424,0.208993,0.158744,0.529901,0.546505,2023-03-01 00:02:00
4.865,21.016,335.742,124.963,460.705,0.072944,-0.43659,-0.102483,0.017551,0.007149,-0.021681,-0.012936,0.019634,0.732634,-0.535845,-0.273947,-0.124959,0.056438,0.311539,0.465377,0.554022,1.663491,0.152084,0.468778,0.383696,0.618529,0.435326,0.415523,0.356895,1.319538,0.955549,0.789646,1.044941,1.353001,0.72392,0.613462,0.246212,…,-0.322244,0.183687,0.404295,-0.169373,0.386584,-0.314775,-1.024058,-0.09459,0.0,1.735322,0.620057,0.0,-0.094702,0.0,0.0,0.0,0.0,0.69121,0.24193,2.525526,4.292975,0.350791,0.32357,0.753829,0.913363,1.002985,1.928621,1.849608,0.004398,0.172967,0.351832,0.368358,0.208416,0.158524,0.529534,0.357703,2023-03-01 00:03:00
27.158,3.451,98.411,44.407,142.818,0.17382,-0.213489,0.096067,0.215709,0.107133,0.078976,0.087818,0.120426,0.763537,-0.430945,-0.205298,-0.062118,0.117266,0.341493,0.495591,0.584519,1.668419,0.156177,0.472732,0.3871,0.623192,0.439034,0.419868,0.361572,1.324595,0.90546,0.78375,1.047708,1.36188,0.732001,0.622712,0.251095,…,-0.369625,0.192377,0.415438,-0.198976,0.389969,-0.312628,-1.02406,0.162221,0.0,1.712096,0.618703,0.0,-0.091884,0.0,0.0,0.0,0.0,0.691207,0.241928,2.52443,4.306694,0.335599,0.31907,0.747533,0.908904,1.001286,1.927084,1.84895,0.004008,0.167391,0.346066,0.365314,0.207839,0.158304,0.529167,0.362452,2023-03-01 00:04:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

# 2 Pre-Processing / Feature Engineering

In [4]:
start = datetime(2023, 3, 1, 0, 0)
end   = datetime(2024, 2, 29, 23, 59, 59)

features = ['X35', 'X96', 'X113', 'X126', 'X261', 'X539', 'X666', 'X690', 'X696']

X_train = data.filter(
    (pl.col("timestamp") >= pl.lit(start)) & (pl.col("timestamp") <= pl.lit(end))
).select(features)

y_train = data.filter(
    (pl.col("timestamp") >= pl.lit(start)) & (pl.col("timestamp") <= pl.lit(end))
)["label"]

X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_train

X35,X96,X113,X126,X261,X539,X666,X690,X696
f64,f64,f64,f64,f64,f64,f64,f64,f64
0.952759,0.505276,0.297555,0.044389,0.032123,0.534967,0.017391,0.285176,0.323195
0.918275,0.504575,0.287799,0.044143,-0.296541,0.518871,0.017294,0.284781,0.322746
0.919497,0.503875,0.278363,0.043898,-0.314776,0.518776,0.017198,0.284385,0.322298
0.940304,0.487659,0.269237,0.038609,-0.310791,0.51868,0.017103,0.283991,0.321851
0.928362,0.486982,0.260409,0.038395,-0.227896,0.518584,0.017008,0.283596,0.321404
…,…,…,…,…,…,…,…,…
0.48488,0.236676,-0.008619,-1.968042,-0.033275,0.582744,0.051806,0.221968,0.197003
0.474246,0.235205,-0.008336,-1.957139,0.009018,0.514809,0.051519,0.22166,0.19673
0.399799,0.233895,-0.008063,-2.097991,0.453205,0.514073,0.051234,0.221353,0.196457


In [5]:
y_train

label
f64
0.562539
0.533686
0.546505
0.357703
0.362452
…
0.396289
0.328993
0.189909


# 3 Model

In [9]:
from xgboost import XGBRegressor

# XGBoost (GPU)
model = XGBRegressor(
    n_estimators=100,
    max_depth=50,
    min_child_weight=10,
    random_state=42,
    tree_method="gpu_hist",  # Use GPU-accelerated histogram method
    predictor="gpu_predictor",
    n_jobs=-1,
    verbosity=0
)
model.fit(X_train_np, y_train_np)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=50, max_leaves=None,
             min_child_weight=10, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=-1,
             num_parallel_tree=None, predictor='gpu_predictor', ...)

# 4 Prediction

## 4.1 Preprocess Test

In [10]:
test = pl.read_parquet("/kaggle/input/drw-crypto-market-prediction/test.parquet")
test = test.with_row_index("ID", offset=1)

X_test = test.select(features)
X_test

X35,X96,X113,X126,X261,X539,X666,X690,X696
f64,f64,f64,f64,f64,f64,f64,f64,f64
0.182896,-0.503141,-0.525747,0.048048,-0.926325,0.106884,0.00687,0.097442,0.142443
-0.184761,-0.113131,-0.038879,0.160252,-0.648221,-0.285764,1.201871,0.35525,0.37403
0.021453,0.138162,-0.028631,0.14032,-0.684555,0.268831,0.047001,0.07298,0.101647
-0.239907,-0.21356,-0.018559,-0.166544,-0.027811,-0.320389,0.015569,0.106965,0.136899
-0.172207,-0.30978,-0.475387,0.524631,0.56557,-0.350371,0.00702,0.106738,0.17255
…,…,…,…,…,…,…,…,…
0.897521,0.354104,0.522156,1.441838,0.666201,0.661136,0.032955,0.116744,0.1565
0.409159,0.785641,-0.312333,0.651243,-0.597156,0.602733,0.037735,0.128928,0.187096
0.115327,-0.046995,0.346656,-0.023924,0.126803,0.114567,0.006577,0.106683,0.166164


## 4.2 Predict

In [11]:
y_pred = model.predict(X_test.to_numpy())
submission = pl.DataFrame({
    "ID": test["ID"],
    "prediction": y_pred
})
submission

ID,prediction
u32,f32
1,-0.130163
2,-0.170369
3,-0.369448
4,1.775169
5,-0.002192
…,…
538146,1.53191
538147,-0.296009
538148,-0.239737


## 5 Submission

In [12]:
submission.write_csv("/kaggle/working/submission.csv")